<a id = 'content'><a/>
### Content page
___

<a id = 'section_0'><a/>
# 0.0 Function Creation
___

In [1]:
import time
from time import sleep
import requests

import pandas as pd
import numpy as np
import random

pd.set_option('display.max_colwidth' , 300)

In [2]:
def red_scrap(title):
    url = 'https://api.pushshift.io/reddit/search/submission'
    df_load = []
    params = {
        'subreddit': title,
        'size' : 100,
        'before': None
    }
    for i in range(14):
        # Access Reddit API
        res = requests.get(url,params)
        data = res.json()
        posts = data['data']
        
        
        df_new = pd.DataFrame(posts)
        df_load.append(df_new)
        
        # Initiating new time stamp (100th position of the 100 size) for before in params
        params['before'] = df_new['created_utc'][99]
        
        # Extract to CSV
        df = pd.concat(df_load, ignore_index = True)
        df.to_csv(f'{title}.csv')
        time.sleep(20)
        print(f'{i+1} Iterations completed')
        
    return df

In [8]:
def date_conversion(df , column):

    time_value = []
    for value in df[column]:
        time_value.append(time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(value)))
        
    df[column] = time_value
    df[[column]] = df[[column]].astype('datetime64[ns]')
    return df

In [49]:
# df_fakenews = red_scrap('fakenews')

1 Iterations completed
2 Iterations completed
3 Iterations completed
4 Iterations completed
5 Iterations completed
6 Iterations completed
7 Iterations completed
8 Iterations completed
9 Iterations completed
10 Iterations completed
11 Iterations completed
12 Iterations completed
13 Iterations completed
14 Iterations completed


In [4]:
# df_politcal_humor = red_scrap('PoliticalHumor')

1 Iterations completed
2 Iterations completed
3 Iterations completed
4 Iterations completed
5 Iterations completed
6 Iterations completed
7 Iterations completed
8 Iterations completed
9 Iterations completed
10 Iterations completed
11 Iterations completed
12 Iterations completed
13 Iterations completed
14 Iterations completed


<a id = 'section_1'><a/>
# 1.0 Data Exploration
___
[(back to top)](#content)

In [45]:
df_fakenews = pd.read_csv('fakenews.csv')
df_fakenews = df_fakenews[['title' , 'subreddit' , 'created_utc']]

# Changing datetime format
df_fakenews = date_conversion(df_fakenews , 'created_utc')

df_politcal_humor = pd.read_csv('PoliticalHumor.csv')
df_politcal_humor = df_politcal_humor[['title' , 'subreddit' , 'created_utc']]

# Changing datetime format
df_politcal_humor = date_conversion(df_politcal_humor , 'created_utc')

In [46]:
print(f'No. of Fakenews Datasets : {len(df_fakenews)}')
print(f'Shape of Fakenews Datasets : {df_fakenews.shape}')

print(f'No. of Politcal Humors Datasets : {len(df_politcal_humor)}')
print(f'Shape of political Humors Datasets : {df_politcal_humor.shape}')

No. of Fakenews Datasets : 1400
Shape of Fakenews Datasets : (1400, 3)
No. of Politcal Humors Datasets : 1400
Shape of political Humors Datasets : (1400, 3)


### 1.0 Checking for Duplicates and Null
___

In [47]:
print(f'No. of Duplicate Cell : {df_fakenews.duplicated().sum()}')
print(f'No. of Null Cell : {df_fakenews.isnull().sum().sum()}')

No. of Duplicate Cell : 0
No. of Null Cell : 0


### 1.1 Checking for Data Leakage
___

In [48]:
df_fakenews['title'].str.contains('fake%').value_counts()

False    1400
Name: title, dtype: int64

In [27]:
df_politcal_humor['title'].str.contains('humor|politic|fun|laugh').value_counts()

False    1378
True       22
Name: title, dtype: int64

In [37]:
df_fakenews = df_fakenews.loc[~df_fakenews['title'].str.contains('fake')]
df_fakenews.head()

,title,subreddit,created_utc
0,Magic in a LIVE Broadcast ABC have to see!,fakenews,2021-08-28 20:52:06
1,ABC anchor nominated for a Pulitzer,fakenews,2021-08-26 23:00:36
2,Never forget MK Ultra,fakenews,2021-08-25 22:16:04
3,Actual Story Behind the Men Who Stare at Goats,fakenews,2021-08-24 23:37:24
5,Taliban “declaration of Emirate”,fakenews,2021-08-21 21:54:16


In [30]:
df_politcal_humor = df_politcal_humor.loc[~df_politcal_humor['title'].str.contains('humor|politic|fun|laugh')]
df_politcal_humor.head()

,title,subreddit,created_utc
0,Next National Building Project.,PoliticalHumor,2021-09-02 20:52:36
1,I can't believe people accuse the GOP of being do as I say not as I do hypocrites who only care about taking away the rights of others,PoliticalHumor,2021-09-02 20:40:02
2,I’m starting to think someone is just making stuff now up to see how many dumb conservatives they can get rid of,PoliticalHumor,2021-09-02 20:27:48
3,"Let's all send wire coat hangers to the Texas State capital: 1100 Congress Ave, Austin, TX 78701.",PoliticalHumor,2021-09-02 20:27:19
4,I would tell them it's ironic but they wouldn't know what that means,PoliticalHumor,2021-09-02 20:25:19


### 1.2 Visualing Some Random Text
___

In [38]:
len(df_fakenews)

1218

In [33]:
random_sentences = random.sample(df_fakenews['title'].to_list() , 10)
for index , sentence in enumerate(random_sentences):
    print(index , sentence)

0 MY NEWS PUNCH F. DEJAHANG ➜⇢➤27-02-2020 ➽ ➜⇢➤Mainstream Vs. Alternative Media Page 11 MAIN FOLDER ➜⇢ ➽ ➤ https://www.edocr.com/user/drdejahang02...
1 Nigel Farage Fake News Lies - Nigel Green deVere CEO
2 Imagine my shock.
3 It's CNN, goes without saying... but they tried to smear Musk and get destroyed on Twitter.
4 Can someone find a news article that complains about the iPhone being a Russian terrorist device, how the Sonic comics scared my some for life, how Minecraft promotes pedofile, how kids are killing people at school because of Nintendo or how Disney is the devil
5 I made this in MS Paint, and deep fried it a little bit.
6 It's the same picture
7 The Artificial Racial Divide with Thomas Sowell
8 DEBUNKED: Why this image is not from California's Creek Fire
9 Detecting Deep Fakes with a Heartbeat


In [29]:
random_sentences = random.sample(df_politcal_humor['title'].to_list() , 10)
for sentence in random_sentences:
    print(sentence)

Taxation is theft at gunpoint and roads are literally suppressing my free speech
they have more in common then you might think
New model from the Office of the President, Ida will hit Florida
Don't lie
Time for a good Belt!
If "big pharma" is just out to make money ...?
Seems strange to me.
Fake headline, but pretty sure that it’s a prediction:
"I would build a great wall, and nobody builds walls better than me, believe me, and I’ll build them very inexpensively. I will build a great great wall on our southern border and I’ll have Mexico pay for that wall." - Donald Trump's Presidential campaign announcement speech - June 16th, 2015
When Wokes and Racists Actually Agree on Everything
